In [9]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# ===== 1. Đọc dữ liệu =====
df = pd.read_csv(r'D:\MINI_Project\DoAn3\perdict_train_XGBoost\students_grading_dataset_clean.csv')

# ===== 2. Loại bỏ các cột không cần thiết =====
drop_cols = ["Student_ID", "First_Name", "Last_Name", "Email", "Grade"]
df = df.drop(columns=drop_cols, errors='ignore')

# ===== 3. Chia X và y =====
y = df['Total_Score']
X = df.drop(columns=['Total_Score'])

# ===== 4. Cột phân loại =====
cat_features = X.select_dtypes(include='object').columns.tolist()

# ===== 5. Chia dữ liệu Train/Test =====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ===== 6. Tạo Pool =====
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# ===== 7. Khởi tạo mô hình CatBoost đã tinh chỉnh =====
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=5,
    loss_function='RMSE',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50
)

# ===== 8. Huấn luyện =====
model.fit(train_pool, eval_set=test_pool, use_best_model=True)

# ===== 9. Dự đoán trên tập test =====
y_pred = model.predict(X_test)

# ===== 10. Đánh giá mô hình =====
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R²:", r2_score(y_test, y_pred))

# ===== 11. Độ quan trọng đặc trưng =====
feature_importances = model.get_feature_importance(prettified=True)
print("\nFeature Importance:")
print(feature_importances)

# ===== 12. Hàm gợi ý lời khuyên học tập =====
def give_advice(student):
    advice_list = []

    if student["Sleep_Hours_per_Night"] < 6:
        advice_list.append("Bạn nên ngủ đủ 7-8 tiếng mỗi ngày để tăng khả năng tập trung và ghi nhớ.")

    if student["Study_Hours_per_Week"] < 10:
        advice_list.append("Hãy dành ít nhất 10-12 giờ mỗi tuần cho việc tự học và ôn tập.")

    if student["Attendance (%)"] < 80:
        advice_list.append("Cần đi học đầy đủ hơn để không bỏ lỡ nội dung quan trọng.")

    if student["Assignments_Avg"] < 70:
        advice_list.append("Hãy cải thiện điểm bài tập bằng cách luyện thêm và nhờ sự hỗ trợ từ giảng viên.")

    if student["Stress_Level (1-10)"] > 7:
        advice_list.append("Bạn nên nghỉ ngơi, tập thể dục hoặc thiền để giảm căng thẳng.")

    if student["Extracurricular_Activities"] == "No":
        advice_list.append("Tham gia hoạt động ngoại khóa để phát triển kỹ năng mềm và mở rộng mối quan hệ.")

    return advice_list

# ===== 13. Ví dụ sinh viên mới =====
new_student = pd.DataFrame([{
    "Gender": "Male",
    "Age": 20,
    "Department": "CS",
    "Attendance (%)": 65,
    "Midterm_Score": 78,
    "Final_Score": 82,
    "Assignments_Avg": 60,
    "Quizzes_Avg": 80,
    "Participation_Score": 8,
    "Projects_Score": 90,
    "Study_Hours_per_Week": 8,
    "Extracurricular_Activities": "No",
    "Internet_Access_at_Home": "Yes",
    "Parent_Education_Level": "Bachelor's",
    "Family_Income_Level": "Medium",
    "Stress_Level (1-10)": 8,
    "Sleep_Hours_per_Night": 4
}])

# ===== 14. Dự đoán & đưa lời khuyên =====
predicted_score = model.predict(new_student)[0]
print(f"\nDự đoán Total_Score: {predicted_score:.2f}")

advice = give_advice(new_student.iloc[0])
print("\nLời khuyên cải thiện hiệu suất học tập:")
for tip in advice:
    print("-", tip)


0:	learn: 6.9624735	test: 7.0702851	best: 7.0702851 (0)	total: 77.3ms	remaining: 1m 17s
100:	learn: 0.6798954	test: 0.7400228	best: 0.7400228 (100)	total: 11.8s	remaining: 1m 45s
200:	learn: 0.2916375	test: 0.3477984	best: 0.3477984 (200)	total: 17.8s	remaining: 1m 10s
300:	learn: 0.2328139	test: 0.2851944	best: 0.2851944 (300)	total: 23.2s	remaining: 53.8s
400:	learn: 0.1991730	test: 0.2552136	best: 0.2552136 (400)	total: 28.8s	remaining: 43s
500:	learn: 0.1770935	test: 0.2380916	best: 0.2380916 (500)	total: 35.8s	remaining: 35.6s
600:	learn: 0.1638464	test: 0.2279460	best: 0.2279460 (600)	total: 43.6s	remaining: 29s
700:	learn: 0.1503379	test: 0.2193131	best: 0.2193131 (700)	total: 50.1s	remaining: 21.4s
800:	learn: 0.1392789	test: 0.2122671	best: 0.2122671 (800)	total: 58s	remaining: 14.4s
900:	learn: 0.1289839	test: 0.2063097	best: 0.2063097 (900)	total: 1m 5s	remaining: 7.22s
999:	learn: 0.1205572	test: 0.2018480	best: 0.2018354 (998)	total: 1m 13s	remaining: 0us

bestTest = 0.201

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Đọc dữ liệu từ file CSV
df = pd.read_csv(r'D:\MINI_Project\DoAn3\students_grading_dataset_final.csv')

# Drop các cột không cần thiết
drop_cols = ["Student_ID", "First_Name", "Last_Name", "Email"]
df = df.drop(columns=drop_cols)

# Target variable
target = "Total_Score"
X = df.drop(columns=[target])
y = df[target]

# Các cột categorical cho CatBoost
cat_features = ["Gender", "Department", "Extracurricular_Activities", "Internet_Access_at_Home", "Parent_Education_Level", "Family_Income_Level"]

# Chia dữ liệu train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tạo Pool cho CatBoost
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

# Huấn luyện mô hình CatBoostRegressor
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='RMSE',
    random_seed=42,
    verbose=100
)
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)

# Đánh giá mô hình
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

# Lưu mô hình
joblib.dump(model, 'catboost_model_final.pkl')
print("Mô hình đã được lưu vào 'catboost_model_final.pkl'")

0:	learn: 6.9586978	test: 7.0666290	best: 7.0666290 (0)	total: 54.8ms	remaining: 54.7s
100:	learn: 0.6454624	test: 0.7103282	best: 0.7103282 (100)	total: 6s	remaining: 53.4s
200:	learn: 0.2709714	test: 0.3262424	best: 0.3262424 (200)	total: 12.2s	remaining: 48.6s
300:	learn: 0.2198798	test: 0.2712121	best: 0.2712121 (300)	total: 17.9s	remaining: 41.7s
400:	learn: 0.1908615	test: 0.2458408	best: 0.2458408 (400)	total: 23.5s	remaining: 35.1s
500:	learn: 0.1720942	test: 0.2320535	best: 0.2320535 (500)	total: 29s	remaining: 28.8s
600:	learn: 0.1570306	test: 0.2219730	best: 0.2219730 (600)	total: 34.3s	remaining: 22.7s
700:	learn: 0.1431941	test: 0.2139541	best: 0.2139541 (700)	total: 39.7s	remaining: 16.9s
800:	learn: 0.1322949	test: 0.2080238	best: 0.2080201 (799)	total: 44.9s	remaining: 11.2s
900:	learn: 0.1222939	test: 0.2024650	best: 0.2024643 (899)	total: 50.1s	remaining: 5.5s
999:	learn: 0.1134442	test: 0.1976428	best: 0.1976428 (999)	total: 56.9s	remaining: 0us

bestTest = 0.1976427